In [46]:
# import necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib as plt
git_url = 'https://github.com/knnthfrmn/dsc25_group.git'

In [ ]:
# load all data

unemployment_data = pd.read_csv('data/global_unemployment_data.csv')
happiness_2015 = pd.read_csv('data/happiness data/2015.csv')
happiness_2016 = pd.read_csv('data/happiness data/2016.csv')
happiness_2017 = pd.read_csv('data/happiness data/2017.csv')
happiness_2018 = pd.read_csv('data/happiness data/2018.csv')
happiness_2019 = pd.read_csv('data/happiness data/2019.csv')

# add year column to happiness dataframes

happiness_2015['year'] = 2015
happiness_2016['year'] = 2016
happiness_2017['year'] = 2017
happiness_2018['year'] = 2018
happiness_2019['year'] = 2019


array(['Finland', 'Denmark', 'Norway', 'Iceland', 'Netherlands',
       'Switzerland', 'Sweden', 'New Zealand', 'Canada', 'Austria',
       'Australia', 'Costa Rica', 'Israel', 'Luxembourg',
       'United Kingdom', 'Ireland', 'Germany', 'Belgium', 'United States',
       'Czech Republic', 'United Arab Emirates', 'Malta', 'Mexico',
       'France', 'Taiwan', 'Chile', 'Guatemala', 'Saudi Arabia', 'Qatar',
       'Spain', 'Panama', 'Brazil', 'Uruguay', 'Singapore', 'El Salvador',
       'Italy', 'Bahrain', 'Slovakia', 'Trinidad & Tobago', 'Poland',
       'Uzbekistan', 'Lithuania', 'Colombia', 'Slovenia', 'Nicaragua',
       'Kosovo', 'Argentina', 'Romania', 'Cyprus', 'Ecuador', 'Kuwait',
       'Thailand', 'Latvia', 'South Korea', 'Estonia', 'Jamaica',
       'Mauritius', 'Japan', 'Honduras', 'Kazakhstan', 'Bolivia',
       'Hungary', 'Paraguay', 'Northern Cyprus', 'Peru', 'Portugal',
       'Pakistan', 'Russia', 'Philippines', 'Serbia', 'Moldova', 'Libya',
       'Montenegro', 'Tajikis